## References

* [MNIST](http://yann.lecun.com/exdb/mnist/)
* [Deep MNIST for experts](https://www.tensorflow.org/get_started/mnist/pros)

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist.input_data import read_data_sets as load_data

np.random.seed(0)
tf.set_random_seed(0)

In [ ]:
CLASS_COUNT = 10
IMAGE_SIZE = 28

data = load_data('data', one_hot=True)

print('Training: {}'.format(data.train.num_examples))
print('Validation: {}'.format(data.validation.num_examples))
print('Testing: {}'.format(data.test.num_examples))

In [ ]:
tf.reset_default_graph()

class Model:
    def __init__(self, x, y):
        self.x, self.y = x, y
        with tf.name_scope('model'):
            W = tf.Variable(tf.zeros([IMAGE_SIZE * IMAGE_SIZE, CLASS_COUNT]), name='weight')
            b = tf.Variable(tf.zeros([CLASS_COUNT]), name='bias')
            self.y_logit = tf.matmul(x, W) + b

class Objective:
    def __init__(self, model, **arguments):
        with tf.name_scope('objective'):
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                labels=model.y, logits=model.y_logit), name='loss')
            optimizer = tf.train.AdamOptimizer(**arguments)
            self.train = optimizer.minimize(self.loss)

class Experiment:
    def run(self, data, model, objective, session, batch_size=2**6, step_count=2**10):
        for _ in range(step_count):
            batch = data.train.next_batch(batch_size)
            feed = {model.x: batch[0], model.y: batch[1]}
            session.run(objective.train, feed_dict=feed)
        feed = {model.x: data.test.images, model.y: data.test.labels}
        y_score = session.run(tf.nn.softmax(model.y_logit), feed_dict=feed)
        y_predicted = np.argmax(y_score, axis=1)
        y_test = np.argmax(data.test.labels, axis=1)
        test_count = data.test.num_examples
        return {
            'Accuracy': (y_test == y_predicted).sum() / test_count,
        }

graph = tf.Graph()
with graph.as_default():
    model = Model(
       tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE * IMAGE_SIZE], name='input'),
       tf.placeholder(tf.float32, shape=[None, CLASS_COUNT], name='output'),
    )
    objective = Objective(model)
    initialize = tf.global_variables_initializer()

with tf.Session(graph=graph) as session:
    with tf.summary.FileWriter('/tmp/model', graph) as writer:
        session.run(initialize)
        experiment = Experiment()
        display(experiment.run(data, model, objective, session))